In [2]:
import torch
import tqdm as notebook_tqdm
import json
import csv

import gensim
from gensim.models import Word2Vec

import numpy as np
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Aaron\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
#Open Training Dataset
file =  open("msr_paraphrase_train.txt", "r", encoding='utf-8')
file_temp = file.readlines()
training_raw_list = []
for line in file_temp:
    training_raw_list.append(line.split('\t'))

In [7]:
#Training data initial processing
training_list1 = []
training_list2 = []
training_labels = []
training_labels_rev = []
for i in range(1, len(training_raw_list)):
    temp_label = training_raw_list[i][0]
    if int(temp_label) == 0:
        training_labels.append(0)
        training_labels_rev.append(1)
    elif int(temp_label) == 1:
        training_labels.append(1)
        training_labels_rev.append(0)
        
    temp_sent1 = training_raw_list[i][3]
    temp_sent2 = training_raw_list[i][4]
    temp_sent2 = temp_sent2.replace("\n", "")
    training_list1.append(temp_sent1)
    training_list2.append(temp_sent2)


In [8]:
print(len(training_list1))
print(len(training_list2))
print(len(training_labels))

4076
4076
4076


In [9]:
#Tokenize training lists using NLTK package
training_tokenized1 = []
training_tokenized2 = []

for i in training_list1:
    temp = word_tokenize(i)
    training_tokenized1.append(temp)
for i in training_list2:
    temp = word_tokenize(i)
    training_tokenized2.append(temp)

In [10]:
#Remove common punctuation tokenized training lists
punct_list = [",", "?", ".", "!", "(", ")", "[", "]", ";", '``', "''", "<", ">", "'", "-"]
for i in range(len(training_tokenized1)):
    temp = training_tokenized1[i]
    temp2 = []
    for j in temp:
        if j not in punct_list:
            temp2.append(j)
    training_tokenized1[i] = temp2
for i in range(len(training_tokenized2)):
    temp = training_tokenized2[i]
    temp2 = []
    for j in temp:
        if j not in punct_list:
            temp2.append(j)
    training_tokenized2[i] = temp2       
    

In [11]:
#Open Test Dataset
file =  open("msr_paraphrase_test.txt", "r", encoding='utf-8')
file_temp = file.readlines()
test_raw_list = []
for line in file_temp:
    test_raw_list.append(line.split('\t'))
file.close()

In [13]:
#Test data initial processing
test_list1 = []
test_list2 = [] 
test_labels = []
test_labels_rev = []
for i in range(1, len(test_raw_list)):
    temp_label = test_raw_list[i][0]
    if int(temp_label) == 0:
        test_labels.append(0)
        test_labels_rev.append(1)
    elif int(temp_label) == 1:
        test_labels.append(1)
        test_labels_rev.append(0)
        
    temp_sent1 = test_raw_list[i][3]
    temp_sent2 = test_raw_list[i][4]
    temp_sent2 = temp_sent2.replace("\n", "")
    test_list1.append(temp_sent1)
    test_list2.append(temp_sent2)


In [14]:
print(len(test_list1))
print(len(test_list2))
print(len(test_labels))

1725
1725
1725


In [15]:
#Tokenize test lists
test_tokenized1 = []
test_tokenized2 = []

for i in test_list1:
    temp = word_tokenize(i)
    test_tokenized1.append(temp)
for i in test_list2:
    temp = word_tokenize(i)
    test_tokenized2.append(temp)

In [16]:
#Remove common punctuation tokenized test lists
punct_list = [",", "?", ".", "!", "(", ")", "[", "]", ";", '``', "''", "<", ">", "'", "-"]
for i in range(len(test_tokenized1)):
    temp = test_tokenized1[i]
    temp2 = []
    for j in temp:
        if j not in punct_list:
            temp2.append(j)
    test_tokenized1[i] = temp2
for i in range(len(test_tokenized2)):
    temp = test_tokenized2[i]
    temp2 = []
    for j in temp:
        if j not in punct_list:
            temp2.append(j)
    test_tokenized2[i] = temp2     

In [17]:
#Determine max sentence lengths
max_sen_length_train = 0
max_sen_length_test = 0

for i in training_tokenized1:
    x = len(i)
    if x > max_sen_length_train:
        max_sen_length_train = x
for i in training_tokenized2:
    x = len(i)
    if x > max_sen_length_train:
        max_sen_length_train = x 
for i in test_tokenized1:
    x = len(i)
    if x > max_sen_length_test:
        max_sen_length_test = x
for i in test_tokenized2:
    x = len(i)
    if x > max_sen_length_test:
        max_sen_length_test = x


In [18]:
max_sen_length_train

34

In [19]:
max_sen_length_test

32

In [20]:
#Combine train and test words to create training list for Word2Vec
word2vec_masterlist = []
for line in training_tokenized1:
    word2vec_masterlist.append(line)
for line in training_tokenized2:
    word2vec_masterlist.append(line)
for line in test_tokenized1:
    word2vec_masterlist.append(line)
for line in test_tokenized2:
    word2vec_masterlist.append(line)

In [21]:
len(word2vec_masterlist)

11602

In [22]:
#Train Word2Vec model
wv_model = gensim.models.Word2Vec(word2vec_masterlist, vector_size=50, workers=4, min_count=1)

In [23]:
train_tensors1 = []
train_tensors2 = []

zero_mask = torch.zeros(50)
zero_mask = torch.reshape(zero_mask, (-1,1))

In [24]:
#Create training sentence tensors. Pad arrays to match maximum dimensions
for sen in training_tokenized1:
    temp_list = []
    for word in sen:
        w_vec = torch.tensor(wv_model.wv[word])
        w_vec = torch.reshape(w_vec, (-1,1))
        temp_list.append(w_vec)
    if len(temp_list) < max_sen_length_train:
        for i in range(len(temp_list), max_sen_length_train):
            temp_list.append(zero_mask)
    temp_stack = torch.stack(temp_list, dim = 0)
    temp_stack = torch.squeeze(temp_stack)
    temp_stack = torch.unsqueeze(temp_stack, 0)
    train_tensors1.append(temp_stack)
for sen in training_tokenized2:
    temp_list = []
    for word in sen:
        w_vec = torch.tensor(wv_model.wv[word])
        w_vec = torch.reshape(w_vec, (-1,1))
        temp_list.append(w_vec)
    if len(temp_list) < max_sen_length_train:
        for i in range(len(temp_list), max_sen_length_train):
            temp_list.append(zero_mask)
    temp_stack = torch.stack(temp_list, dim = 0)
    temp_stack = torch.squeeze(temp_stack)
    temp_stack = torch.unsqueeze(temp_stack, 0)
    train_tensors2.append(temp_stack)

In [86]:
test_tensors1 = []
test_tensors2 = []

In [87]:
#Create test sentence tensors
for sen in test_tokenized1:
    temp_list = []
    for word in sen:
        w_vec = torch.tensor(wv_model.wv[word])
        w_vec = torch.reshape(w_vec, (-1,1))
        temp_list.append(w_vec)
    if len(temp_list) < max_sen_length_test:
        for i in range(len(temp_list), max_sen_length_test):
            temp_list.append(zero_mask)
    temp_stack = torch.stack(temp_list, dim = 0)
    temp_stack = torch.squeeze(temp_stack)
    temp_stack = torch.unsqueeze(temp_stack, 0)
    test_tensors1.append(temp_stack)
for sen in test_tokenized2:
    temp_list = []
    for word in sen:
        w_vec = torch.tensor(wv_model.wv[word])
        w_vec = torch.reshape(w_vec, (-1,1))
        temp_list.append(w_vec)
    if len(temp_list) < max_sen_length_test:
        for i in range(len(temp_list), max_sen_length_test):
            temp_list.append(zero_mask)
    temp_stack = torch.stack(temp_list, dim = 0)
    temp_stack = torch.squeeze(temp_stack)
    temp_stack = torch.unsqueeze(temp_stack, 0)
    test_tensors2.append(temp_stack)

In [26]:
#Create Training Master Sets
train_master_set1 = torch.stack(train_tensors1)
train_master_set2 = torch.stack(train_tensors2)

In [27]:
train_master_set1.shape

torch.Size([4076, 1, 34, 50])

In [28]:
#Create Training Master Labels
train_master_labels = torch.tensor(training_labels).float()
train_master_labels_rev = torch.tensor(training_labels_rev).float()

In [29]:
#Create Training Set Subsets
train_set1 = []
train_set2 = []
split_train1 = np.array_split(train_tensors1, 64)
split_train2 = np.array_split(train_tensors2, 64)
for i in range(64):
    temp1 = torch.stack(list(split_train1[i]))
    train_set1.append(temp1)
    temp2 = torch.stack(list(split_train2[i]))
    train_set2.append(temp2)

C:\Users\Aaron\Desktop\DLH\Project\Python\penv\lib\site-packages\numpy\core\fromnumeric.py:43: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  result = getattr(asarray(obj), method)(*args, **kwds)
C:\Users\Aaron\Desktop\DLH\Project\Python\penv\lib\site-packages\numpy\core\fromnumeric.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = getattr(asarray(obj), m

In [32]:
train_set1[0].shape

torch.Size([64, 1, 34, 50])

In [33]:
#Create Train Label Sets
train_label_sets = []
split_train_labels = np.array_split(training_labels, 64)
for i in split_train_labels:
    train_label_sets.append(torch.tensor(i).float())


In [88]:
#Create Test Sets. Batch size = 27
test_set1 = []
test_set2 = []
split_test1 = np.array_split(test_tensors1, 27)
split_test2 = np.array_split(test_tensors2, 27)
for i in range(27):
    temp1 = torch.stack(list(split_test1[i]))
    test_set1.append(temp1)
    temp2 = torch.stack(list(split_test2[i]))
    test_set2.append(temp2)

In [89]:
#Create Test Master Sets
test_master_set1 = torch.stack(test_tensors1)
test_master_set2 = torch.stack(test_tensors2)

In [90]:
test_set1[0].shape

torch.Size([64, 1, 32, 50])

In [91]:
#Create Test Master Labels
test_master_labels = torch.tensor(test_labels).float()
test_master_labels_rev = torch.tensor(test_labels_rev).float()

In [92]:
#Create Test Label Sets
test_label_sets = []
split_test_labels = np.array_split(test_labels, 27)
for i in split_test_labels:
    test_label_sets.append(torch.tensor(i).float())

In [40]:
#Import Pytorch Neural Network Packages

import os
import random
import torch
import torch.nn as nn
import torch.nn.functional as F

In [41]:
from numpy import array  
from numpy.linalg import norm

In [42]:
#K-max Pooling Code

def kmax_pooling(x, dim, k):
    index = torch.topk(x, k, dim = dim)[1].sort(dim = dim)[0]
    return x.gather(dim, index)

In [191]:
class CNN1(nn.Module):
    def __init__(self):
        super(CNN1, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 5, 3, padding = 2)
        self.conv2 = nn.Conv2d(5, 25, 3, padding = 2)
        self.pool = nn.MaxPool2d(3)
        self.fc1 = nn.Linear(4050, 500)
        self.fc2 = nn.Linear(500, 25)
        self.pwd = nn.PairwiseDistance(p=1)
        self.cos = nn.CosineSimilarity(dim=1, eps=1e-08)

    def forward(self, x1, x2):

        x1 = F.relu(self.conv1(x1))
        x1 = kmax_pooling(x1, 2, 3)

        x1 = F.relu(self.conv2(x1))
        x1 = kmax_pooling(x1, 2, 3)
        
        x1 = torch.flatten(x1, 1)
        x1 = self.fc1(x1)
        x1 = self.fc2(x1)
        
        
        x2 = F.relu(self.conv1(x2))
        x2 = kmax_pooling(x2, 2, 3)

        x2 = F.relu(self.conv2(x2))
        x2 = kmax_pooling(x2, 2, 3)    
        
        x2 = torch.flatten(x2, 1)
        x2 = self.fc1(x2)
        x2 = self.fc2(x2)
        
        
        result1 = self.pwd(x1, x2)
        
        result2 = torch.exp(-result1)

        return result2

In [192]:
#Create Instance of CNN Model

model = CNN1()

#Define Evaluation Criterion as Mean Square Error Loss

criterion = nn.MSELoss()

#Define Optimizer as Stochastic Gradient Descent

optimizer = torch.optim.SGD(model.parameters(), lr = np.exp(-1))

In [193]:
#Model Training Code

n_epochs = 15
def train_model(model, train_data1, train_data2, training_labels, n_epoch=n_epochs, optimizer=optimizer, criterion=criterion):
    import torch.optim as optim

    model.train()
    
    
    for epoch in range(n_epoch):
        curr_epoch_loss = []
        for i in range(64):
            train_temp1 = train_data1[i]
            train_temp2 = train_data2[i]
            
            labels_temp = training_labels[i]
            
            y_hat = model(train_temp1, train_temp2)
            
            y_hat = torch.reshape(y_hat, (-1,1))

            labels_temp = torch.reshape(labels_temp, (-1,1))
        
            loss = criterion(y_hat, labels_temp)
            
            optimizer.zero_grad()
            
            loss.backward()
            
            optimizer.step()
        
            curr_epoch_loss.append(loss.cpu().data.numpy())
        
        print(f"Epoch {epoch}: curr_epoch_loss={np.mean(curr_epoch_loss)}")

    return model

In [194]:
#Train Model

seed = 24
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

model = train_model(model, train_set1, train_set2, train_label_sets)

Epoch 0: curr_epoch_loss=0.2101428210735321
Epoch 1: curr_epoch_loss=0.1986837387084961
Epoch 2: curr_epoch_loss=0.1951833814382553
Epoch 3: curr_epoch_loss=0.18699932098388672
Epoch 4: curr_epoch_loss=0.18574273586273193
Epoch 5: curr_epoch_loss=0.18368230760097504
Epoch 6: curr_epoch_loss=0.17737165093421936
Epoch 7: curr_epoch_loss=0.17418533563613892
Epoch 8: curr_epoch_loss=0.1728201061487198
Epoch 9: curr_epoch_loss=0.16691292822360992
Epoch 10: curr_epoch_loss=0.1655031144618988
Epoch 11: curr_epoch_loss=0.16205278038978577
Epoch 12: curr_epoch_loss=0.16143618524074554
Epoch 13: curr_epoch_loss=0.158069908618927
Epoch 14: curr_epoch_loss=0.15928170084953308


In [195]:
#Model Testing Code
def eval_model(model, test_data1, test_data2, target_test):

    model.eval()
    Y_pred = []
    Y_test = []
      
    y_hat = model(test_data1, test_data2)
        
    y_pred = (y_hat > 0.5).type(torch.float)
    
    Y_pred = torch.reshape(y_pred, (-1,1))

    Y_test = torch.reshape(target_test, (-1,1))
        

    return Y_pred, Y_test

In [196]:
import sklearn
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [197]:
#Test Model and Calculate Accuracy and F1 Score

y_pred, y_true = eval_model(model, test_master_set1, test_master_set2, test_master_labels)
acc = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
print(("Validation Accuracy: " + str(acc)))
print(("Validation F1: " + str(f1)))

Validation Accuracy: 0.7049275362318841
Validation F1: 0.8098617855808741
